In [1]:
!nvidia-smi

Sun May 19 15:01:03 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   63C    P0             22W /   90W |       0MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
from torch import nn

In [3]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [4]:
torch.cuda.device_count()

1

In [6]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [7]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [8]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [9]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1161, 0.0249, 0.6990],
        [0.5830, 0.2954, 0.4553]])

In [10]:
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.1161, 0.0249, 0.6990],
        [0.5830, 0.2954, 0.4553]])
tensor([[0.1161, 0.0249, 0.6990],
        [0.5830, 0.2954, 0.4553]], device='cuda:0')


In [11]:
X+Z

tensor([[1.1161, 1.0249, 1.6990],
        [1.5830, 1.2954, 1.4553]], device='cuda:0')

In [12]:
Z.cuda(0) is Z

True

In [13]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [14]:
net(X)

tensor([[-0.3822],
        [-0.3822]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [15]:
net[0].weight.data

tensor([[-0.0514, -0.4088, -0.4135]], device='cuda:0')

In [16]:
net[0].weight.data.device

device(type='cuda', index=0)